In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from keras import Input
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Softmax, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
import tensorflow as tf
import os
import json
import cv2
from sklearn.model_selection import train_test_split

In [21]:
FOLDER_NAME = '../data'
OUTPUT_FILENAME = '../data.json'

with open(OUTPUT_FILENAME, 'r') as json_file:
    data = json.load(json_file)

pictures = []
letters = []
unique_groups = set()
for datum in data:
    img = cv2.imread(
        os.path.join(FOLDER_NAME, datum['pic']),
        cv2.IMREAD_GRAYSCALE
    )
    img = img.astype('float32')
    img /= 255
    pictures.append(img)
    letters.append(datum['group'])
    unique_groups.add(datum['group'])
pictures = np.expand_dims(np.array(pictures), axis=3)
group_to_vec = {gr: i for i, gr in enumerate(unique_groups)}
groups = []
for letter in letters:
    vec = np.zeros(len(unique_groups))
    vec[group_to_vec[letter]] = 1
    groups.append(vec)
groups = np.array(groups)

p_train, p_test, l_train, l_test = train_test_split(pictures, groups, test_size=0.2)

In [22]:
input_shape = (32,32,1)
output_shape = len(unique_groups)
pool_size = (2, 2)
conv_filters1 = 32
conv_filters2 = 64
conv_kernel = 3
print(output_shape)

12


In [23]:
model = Sequential(name='CnnToSmiles')

model.add(Input(shape=(32,32,1)))
model.add(Conv2D(filters=conv_filters1, kernel_size=conv_kernel, strides=(1, 1), padding="same"))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Conv2D(filters=conv_filters2, kernel_size=conv_kernel, strides=(1, 1), padding="same"))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(len(unique_groups), activation='softmax'))

model.summary()

opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "CnnToSmiles"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               2097664   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               

In [24]:
model.fit(p_train, l_train, validation_data=(p_test, l_test), batch_size=32, epochs=20)

Epoch 1/20
88/88 [==============================] - 3s 33ms/step - loss: 0.9653 - accuracy: 0.7304 - val_loss: 0.5444 - val_accuracy: 0.8529
Epoch 2/20
88/88 [==============================] - 3s 31ms/step - loss: 0.4747 - accuracy: 0.8700 - val_loss: 0.4250 - val_accuracy: 0.8714
Epoch 3/20
88/88 [==============================] - 3s 30ms/step - loss: 0.3740 - accuracy: 0.8921 - val_loss: 0.3763 - val_accuracy: 0.8857
Epoch 4/20
88/88 [==============================] - 3s 31ms/step - loss: 0.3202 - accuracy: 0.8989 - val_loss: 0.3417 - val_accuracy: 0.8929
Epoch 5/20
88/88 [==============================] - 3s 33ms/step - loss: 0.2822 - accuracy: 0.9121 - val_loss: 0.3163 - val_accuracy: 0.9000
Epoch 6/20
88/88 [==============================] - 3s 31ms/step - loss: 0.2577 - accuracy: 0.9211 - val_loss: 0.3049 - val_accuracy: 0.9029
Epoch 7/20
88/88 [==============================] - 3s 31ms/step - loss: 0.2381 - accuracy: 0.9243 - val_loss: 0.3011 - val_accuracy: 0.9057
Epoch 8/20
88

In [25]:
model.save('../letter_to_group.h5')